In [1]:
from skorch import NeuralNetRegressor
import torch.optim as optim
import torch
import torch.nn as nn
from torch.nn.functional import relu

import torch
import torch.nn as nn
from torch.nn.functional import relu
import torch
import torch.nn as nn
from torch.nn.functional import relu


class EmbedNet(nn.Module):
    def __init__(
        self,
        num_embeddings,
        embedding_dim,
        num_numeric_features,
        num_outputs,
        hidden_size=128,
        num_layers=2,
    ):
        super(EmbedNet, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        total_feature_dim = (
            embedding_dim + num_numeric_features
        )  # Total input dimension for the first hidden layer

        # First layer after concatenation needs to handle total combined dimensions
        self.fc1 = nn.Linear(total_feature_dim, hidden_size)
        self.fc_layers = nn.ModuleList(
            [nn.Linear(hidden_size, hidden_size) for _ in range(num_layers - 1)]
        )
        self.output = nn.Linear(hidden_size, num_outputs)

    def forward(self, X):
        # Assuming the first column of X is for embedding, and the rest are numerical features
        x_cat = X[:, 0].long()  # Categorical features for embedding
        x_num = X[:, 1:]  # Numerical features

        x_embed = self.embedding(x_cat).view(
            x_cat.size(0), -1
        )  # Reshape embedded output
        x_combined = torch.cat(
            [x_embed, x_num], dim=1
        )  # Concatenate embeddings with numerical features

        x = relu(self.fc1(x_combined))
        for layer in self.fc_layers:
            x = relu(layer(x))
        x = self.output(x)
        return x


from skorch import NeuralNetRegressor

net = NeuralNetRegressor(
    module=EmbedNet,
    module__num_embeddings=10,
    module__embedding_dim=5,
    module__num_numeric_features=8,
    module__num_outputs=1,
    criterion=torch.nn.MSELoss,
    optimizer=torch.optim.Adam,
    optimizer__lr=0.01,
    max_epochs=10,
    batch_size=128,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

# Assuming X_cat and X_num are previously defined torch tensors
X_cat = torch.randint(
    0, 10, (100000, 1), dtype=torch.long
)  # Random indices for embedding
X_num = torch.randn(100000, 8)  # Random numerical features
X = torch.cat((X_cat, X_num), dim=1).numpy()  # Convert to numpy and concatenate
y = torch.randn(100000, 1).numpy()  # Random targets

# Fit the model
net.fit(X, y)

# Predict using the trained model
y_pred = net.predict(X[:5])
print(y_pred)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.9917        1.0012  0.8254
      2        0.9904        1.0012  0.8415
      3        0.9904        1.0012  0.8539
      4        0.9904        1.0012  0.8639
      5        0.9904        1.0012  0.8053
      6        0.9904        1.0012  0.8536
      7        0.9904        1.0012  0.8350
      8        0.9904        1.0012  0.8565
      9        0.9904        1.0012  0.7985
     10        0.9904        1.0012  0.8222
[[0.05384421]
 [0.05384421]
 [0.05384421]
 [0.05384421]
 [0.05384421]]


# Dynamic

In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from skorch import NeuralNetRegressor


class EmbedNet(nn.Module):
    def __init__(self, input_dim, categorical_indices, categorical_sizes):
        super(EmbedNet, self).__init__()
        # Embedding layers for categorical features
        self.embeddings = nn.ModuleList(
            [
                nn.Embedding(
                    num_embeddings=size, embedding_dim=min(50, (size + 1) // 2)
                )
                for size in categorical_sizes
            ]
        )

        num_embeddings_output = sum(e.embedding_dim for e in self.embeddings)
        num_numeric_features = input_dim - len(categorical_indices)
        total_feature_dim = num_embeddings_output + num_numeric_features

        # Network layers
        hidden_size = 128
        self.fc1 = nn.Linear(total_feature_dim, hidden_size)
        self.fc_layers = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
        )
        self.output = nn.Linear(hidden_size, 1)  # Assuming regression

    def forward(self, X):
        x_cat = [X[:, i].long() for i in range(len(self.embeddings))]
        x_num = X[:, len(self.embeddings) :]

        x_embed = [embedding(cat) for cat, embedding in zip(x_cat, self.embeddings)]
        x_embed = (
            torch.cat(x_embed, dim=1)
            if x_embed
            else torch.empty(X.size(0), 0).to(X.device)
        )

        # Ensure numerical features are the correct tensor type
        x_num = x_num.float()

        # Check dimensions before concatenation
        # print("x_embed shape:", x_embed.shape)
        # print("x_num shape:", x_num.shape)

        x_combined = torch.cat([x_embed, x_num], dim=1)
        x = self.fc1(x_combined)
        x = self.fc_layers(x)
        return self.output(x)


# Function to prepare the network with dynamic features
def prepare_network(X, y):
    categorical_features = [
        X.columns.get_loc(col) for col in X.select_dtypes(include=["category"]).columns
    ]
    categorical_sizes = [
        X[col].nunique() for col in X.select_dtypes(include=["category"]).columns
    ]

    net = NeuralNetRegressor(
        module=EmbedNet,
        module__input_dim=X.shape[1],
        module__categorical_indices=categorical_features,
        module__categorical_sizes=categorical_sizes,
        criterion=torch.nn.MSELoss,
        optimizer=torch.optim.Adam,
        optimizer__lr=0.01,
        max_epochs=10,
        batch_size=32,
        train_split=None,  # Disable cross-validation
        device="cuda" if torch.cuda.is_available() else "cpu",
    )

    return net


def prepare_data(X, categorical_columns):
    # Convert categorical columns to category codes
    for col in categorical_columns:
        X[col] = X[col].astype("category").cat.codes
    return X


# Prepare the network and data
categorical_columns = ["cat1", "cat2"]
X_prepared = prepare_data(X.copy(), categorical_columns)

# Convert DataFrame to numpy array
X_np = X_prepared.values.astype(np.float32)
y_np = y.values.astype(np.float32)

# Initialize and fit the model
net = prepare_network(X_prepared, y)
net.fit(X_np, y_np)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

# Dynamic Improved

In [76]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from skorch import NeuralNetRegressor


import torch
import torch.nn as nn

import torch
import torch.nn as nn


class EmbedNet(nn.Module):
    def __init__(
        self,
        input_dims,
        num_outputs,
        categorical_info=None,
        hidden_size=128,
        num_layers=2,
        activation="relu",
        use_batchnorm=False,
        dropout_prob=0.1,
        y_range=None,
        problem_type="regression",
    ):
        super(EmbedNet, self).__init__()
        self.problem_type = problem_type
        self.y_range = y_range
        self.y_constraint = None
        self.setup_y_constraints()

        # Setup embedding layers if categorical info is provided
        self.embeddings = nn.ModuleList()
        input_size = (
            input_dims - len(categorical_info) if categorical_info else input_dims
        )
        if categorical_info:
            for num_categories, embedding_size in categorical_info:
                self.embeddings.append(nn.Embedding(num_categories, embedding_size))
                input_size += embedding_size  # Adjust input size based on embeddings

        # Setup network layers
        layers = []
        if use_batchnorm:
            layers.append(nn.BatchNorm1d(input_size))
        layers.append(nn.Linear(input_size, hidden_size))
        layers.append(self.get_activation_fn(activation)())
        for _ in range(num_layers - 1):
            if use_batchnorm:
                layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.Dropout(dropout_prob))
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(self.get_activation_fn(activation)())
        layers.append(nn.Linear(hidden_size, num_outputs))
        self.main_block = nn.Sequential(*layers)

    def get_activation_fn(self, name):
        activations = {
            "relu": nn.ReLU,
            "elu": nn.ELU,
            "tanh": nn.Tanh,
            "identity": nn.Identity,
        }
        return activations.get(name, nn.ReLU)

    def setup_y_constraints(self):
        if self.y_range:
            if self.y_range[0] == -float("inf") and self.y_range[1] == float("inf"):
                self.y_constraint = None
            elif self.y_range[0] >= 0 and self.y_range[1] == float("inf"):
                self.y_constraint = "nonnegative"
            elif self.y_range[0] == -float("inf") and self.y_range[1] <= 0:
                self.y_constraint = "nonpositive"
            else:
                self.y_constraint = "bounded"
                self.y_lower, self.y_upper = self.y_range

    def forward(self, x):
        print("Input tensor shape:", x.shape)  # Add debugging 
        # Handle categorical features
        if self.embeddings:
            x_cat = [embed(x[:, i].long()) for i, embed in enumerate(self.embeddings)]
            print("Embedded outputs shapes:", [t.shape for t in x_cat])  # Add debugging
            x = torch.cat(x_cat + [x[:, len(self.embeddings) :]], dim=1)
        print("After embedding/concatenation:", x.shape) # Add debugging
        x = self.main_block(x)
        if self.problem_type == "regression" and self.y_constraint:
            x = self.apply_y_constraint(x)
        return x.view(-1)

    def apply_y_constraint(self, x):
        if self.y_constraint == "nonnegative":
            return torch.abs(x)
        elif self.y_constraint == "nonpositive":
            return -torch.abs(x)
        elif self.y_constraint == "bounded":
            return torch.sigmoid(x) * (self.y_upper - self.y_lower) + self.y_lower
        return x


import torch
from skorch import NeuralNetRegressor

# Example DataFrame with mixed types
import pandas as pd
import numpy as np


def prepare_data(X, categorical_columns, max_indices=None): 
    for col in categorical_columns:
        X[col] = X[col].astype("category").cat.codes + 1
        if max_indices and col in max_indices:
            print(f"Column {col}: Maximum index found: {X[col].max()}")
    return X

data = pd.DataFrame(
    {
        "cat1": pd.Categorical(["a", "b", "a", "c"]),
        "num1": np.random.randn(4),
        "cat2": pd.Categorical([1, 2, 1, 2]),
        "target": np.random.randn(4),
    }
)

X = data.drop(columns=["target"])
y = data["target"]

# Convert DataFrame to numpy array after encoding categorical features
categorical_columns = ["cat1", "cat2"]
max_indices = {
    "cat1": 2,  # Assuming the embedding layer for "cat1" can handle indices 0, 1, 2
    "cat2": 1   # Assuming the embedding layer for "cat2" can handle indices 0, 1
}

# Prepare the data
X_prepared = prepare_data(X.copy(), categorical_columns, max_indices)

# Convert DataFrame to numpy array
X_np = X_prepared.values.astype(np.float32)
y_np = y.values.astype(np.float32)


def prepare_network(X, num_outputs):
    categorical_info = []
    for col in X.select_dtypes(include=["category"]).columns:
        max_index = X[col].cat.codes.max()  # Get the maximum index from codes
        num_categories = max_index + 1  # Account for the number of unique categories
        embedding_size = min(50, (num_categories + 1) // 2)  # Determine the size of embedding
        categorical_info.append((num_categories, embedding_size))
        
    net = NeuralNetRegressor(
        module=EmbedNet,
        module__input_dims=X.shape[1],
        module__num_outputs=num_outputs,
        module__categorical_info=categorical_info,
        criterion=torch.nn.MSELoss,
        optimizer=torch.optim.Adam,
        optimizer__lr=0.01,
        max_epochs=10,
        batch_size=32,
        train_split=None,
        device="cuda" if torch.cuda.is_available() else "cpu",
    )
    return net


# Initialize and fit the model
net = prepare_network(X, 1)  # Assuming 'target' is a single output scenario
try:
    net.fit(X_np, y_np)
except Exception as e:
    print(f"Error during training: {str(e)}")
    # Optionally print more diagnostic information
    print("df:", data)
    print("Input shapes:", X_np.shape)
    print("Sample inputs:", X_np[:5])
    print("Target shapes:", y_np.shape)
    print("Sample targets:", y_np[:5])

Column cat1: Maximum index found: 3
Column cat2: Maximum index found: 2
Input tensor shape: torch.Size([4, 3])
Error during training: index out of range in self
df:   cat1      num1 cat2    target
0    a -2.286832    1 -0.459186
1    b  0.134887    2  0.086403
2    a  0.053830    1 -0.330259
3    c  0.183663    2  0.006849
Input shapes: (4, 3)
Sample inputs: [[ 1.         -2.286832    1.        ]
 [ 2.          0.13488741  2.        ]
 [ 1.          0.05382965  1.        ]
 [ 3.          0.18366256  2.        ]]
Target shapes: (4,)
Sample targets: [-0.4591855   0.08640274 -0.3302594   0.00684865]


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from skorch import NeuralNetRegressor


import torch
import torch.nn as nn

import torch
import torch.nn as nn

def get_embed_sizes(train_dataset, params, num_categs_per_feature):
    """Returns list of embedding sizes for each categorical variable.
    Selects this adaptively based on training_dataset.
    Note: Assumes there is at least one embed feature.
    """
    max_embedding_dim = params["max_embedding_dim"]
    embed_exponent = params["embed_exponent"]
    size_factor = params["embedding_size_factor"]
    embed_dims = [
        int(size_factor * max(2, min(max_embedding_dim, 1.6 * num_categs_per_feature[i] ** embed_exponent))) for i in range(len(num_categs_per_feature))
    ]
    return embed_dims

class EmbedNet(nn.Module):
    def __init__(
        self,
        input_dims,
        num_outputs,
        categorical_info=None,
        hidden_size=128,
        num_layers=2,
        activation="relu",
        use_batchnorm=False,
        dropout_prob=0.1,
        y_range=None,
        problem_type="regression",
    ):
        super(EmbedNet, self).__init__()
        self.problem_type = problem_type
        self.y_range = y_range
        self.y_constraint = None
        self.setup_y_constraints()

        # Setup embedding layers if categorical info is provided
        self.embeddings = nn.ModuleList()
        input_size = (
            input_dims - len(categorical_info) if categorical_info else input_dims
        )
        if categorical_info:
            for num_categories, embedding_size in categorical_info:
                self.embeddings.append(nn.Embedding(num_categories, embedding_size))
                input_size += embedding_size  # Adjust input size based on embeddings

        # Setup network layers
        layers = []
        if use_batchnorm:
            layers.append(nn.BatchNorm1d(input_size))
        layers.append(nn.Linear(input_size, hidden_size))
        layers.append(self.get_activation_fn(activation)())
        for _ in range(num_layers - 1):
            if use_batchnorm:
                layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.Dropout(dropout_prob))
            layers.append(nn.Linear(hidden_size, hidden_size))
            layers.append(self.get_activation_fn(activation)())
        layers.append(nn.Linear(hidden_size, num_outputs))
        self.main_block = nn.Sequential(*layers)

    def get_activation_fn(self, name):
        activations = {
            "relu": nn.ReLU,
            "elu": nn.ELU,
            "tanh": nn.Tanh,
            "identity": nn.Identity,
        }
        return activations.get(name, nn.ReLU)

    def setup_y_constraints(self):
        if self.y_range:
            if self.y_range[0] == -float("inf") and self.y_range[1] == float("inf"):
                self.y_constraint = None
            elif self.y_range[0] >= 0 and self.y_range[1] == float("inf"):
                self.y_constraint = "nonnegative"
            elif self.y_range[0] == -float("inf") and self.y_range[1] <= 0:
                self.y_constraint = "nonpositive"
            else:
                self.y_constraint = "bounded"
                self.y_lower, self.y_upper = self.y_range

    def forward(self, x):
        print("Input tensor shape:", x.shape)  # Add debugging 
        # Handle categorical features
        if self.embeddings:
            x_cat = [embed(x[:, i].long()) for i, embed in enumerate(self.embeddings)]
            print("Embedded outputs shapes:", [t.shape for t in x_cat])  # Add debugging
            x = torch.cat(x_cat + [x[:, len(self.embeddings) :]], dim=1)
        print("After embedding/concatenation:", x.shape) # Add debugging
        x = self.main_block(x)
        if self.problem_type == "regression" and self.y_constraint:
            x = self.apply_y_constraint(x)
        return x.view(-1)

    def apply_y_constraint(self, x):
        if self.y_constraint == "nonnegative":
            return torch.abs(x)
        elif self.y_constraint == "nonpositive":
            return -torch.abs(x)
        elif self.y_constraint == "bounded":
            return torch.sigmoid(x) * (self.y_upper - self.y_lower) + self.y_lower
        return x


import torch
from skorch import NeuralNetRegressor

# Example DataFrame with mixed types
import pandas as pd
import numpy as np


def prepare_data(X, categorical_columns, max_indices=None): 
    for col in categorical_columns:
        X[col] = X[col].astype("category").cat.codes + 1
        if max_indices and col in max_indices:
            print(f"Column {col}: Maximum index found: {X[col].max()}")
    return X

data = pd.DataFrame(
    {
        "cat1": pd.Categorical(["a", "b", "a", "c"]),
        "num1": np.random.randn(4),
        "cat2": pd.Categorical([1, 2, 1, 2]),
        "target": np.random.randn(4),
    }
)

X = data.drop(columns=["target"])
y = data["target"]

# Convert DataFrame to numpy array after encoding categorical features
categorical_columns = ["cat1", "cat2"]
max_indices = {
    "cat1": 2,  # Assuming the embedding layer for "cat1" can handle indices 0, 1, 2
    "cat2": 1   # Assuming the embedding layer for "cat2" can handle indices 0, 1
}

# Prepare the data
X_prepared = prepare_data(X.copy(), categorical_columns, max_indices)

# Convert DataFrame to numpy array
X_np = X_prepared.values.astype(np.float32)
y_np = y.values.astype(np.float32)


def prepare_network(X, num_outputs):
    categorical_info = []
    for col in X.select_dtypes(include=["category"]).columns:
        max_index = X[col].cat.codes.max()  # Get the maximum index from codes
        num_categories = max_index + 1  # Account for the number of unique categories
        embedding_size = min(50, (num_categories + 1) // 2)  # Determine the size of embedding
        categorical_info.append((num_categories, embedding_size))
        
    net = NeuralNetRegressor(
        module=EmbedNet,
        module__input_dims=X.shape[1],
        module__num_outputs=num_outputs,
        module__categorical_info=categorical_info,
        criterion=torch.nn.MSELoss,
        optimizer=torch.optim.Adam,
        optimizer__lr=0.01,
        max_epochs=10,
        batch_size=32,
        train_split=None,
        device="cuda" if torch.cuda.is_available() else "cpu",
    )
    return net


# Initialize and fit the model
net = prepare_network(X, 1)  # Assuming 'target' is a single output scenario
try:
    net.fit(X_np, y_np)
except Exception as e:
    print(f"Error during training: {str(e)}")
    # Optionally print more diagnostic information
    print("df:", data)
    print("Input shapes:", X_np.shape)
    print("Sample inputs:", X_np[:5])
    print("Target shapes:", y_np.shape)
    print("Sample targets:", y_np[:5])

Column cat1: Maximum index found: 3
Column cat2: Maximum index found: 2
Input tensor shape: torch.Size([4, 3])
Error during training: index out of range in self
df:   cat1      num1 cat2    target
0    a -2.286832    1 -0.459186
1    b  0.134887    2  0.086403
2    a  0.053830    1 -0.330259
3    c  0.183663    2  0.006849
Input shapes: (4, 3)
Sample inputs: [[ 1.         -2.286832    1.        ]
 [ 2.          0.13488741  2.        ]
 [ 1.          0.05382965  1.        ]
 [ 3.          0.18366256  2.        ]]
Target shapes: (4,)
Sample targets: [-0.4591855   0.08640274 -0.3302594   0.00684865]
